In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn import svm
from sklearn.metrics import ConfusionMatrixDisplay

In [114]:
crab = pd.read_csv('CrabAgePrediction.csv')
crab.head()

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,F,1.4375,1.1750,0.4125,24.635715,12.332033,5.584852,6.747181,9
1,M,0.8875,0.6500,0.2125,5.400580,2.296310,1.374951,1.559222,6
2,I,1.0375,0.7750,0.2500,7.952035,3.231843,1.601747,2.764076,6
3,F,1.1750,0.8875,0.2500,13.480187,4.748541,2.282135,5.244657,10
4,I,0.8875,0.6625,0.2125,6.903103,3.458639,1.488349,1.700970,6


In [115]:
crab.isnull().sum()

Sex               0
Length            0
Diameter          0
Height            0
Weight            0
Shucked Weight    0
Viscera Weight    0
Shell Weight      0
Age               0
dtype: int64

In [116]:
crab.shape

(3893, 9)

In [117]:
crab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3893 entries, 0 to 3892
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sex             3893 non-null   object 
 1   Length          3893 non-null   float64
 2   Diameter        3893 non-null   float64
 3   Height          3893 non-null   float64
 4   Weight          3893 non-null   float64
 5   Shucked Weight  3893 non-null   float64
 6   Viscera Weight  3893 non-null   float64
 7   Shell Weight    3893 non-null   float64
 8   Age             3893 non-null   int64  
dtypes: float64(7), int64(1), object(1)
memory usage: 273.9+ KB


In [118]:
#crab = pd.get_dummies(crab, columns=['Sex'])

In [119]:
crab["Sex"] = crab["Sex"].map( {"M": 2, "F": 1, "I": 0})

In [120]:
crab.describe()

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
count,3893.000000,3893.000000,3893.000000,3893.000000,3893.000000,3893.000000,3893.000000,3893.000000,3893.000000
mean,1.051888,1.311306,1.020893,0.349374,23.567275,10.207342,5.136546,6.795844,9.954791
std,0.826327,0.300431,0.248233,0.104976,13.891201,6.275275,3.104133,3.943392,3.220967
min,0.000000,0.187500,0.137500,0.000000,0.056699,0.028349,0.014175,0.042524,1.000000
25%,0.000000,1.125000,0.875000,0.287500,12.672227,5.343881,2.664853,3.713785,8.000000
50%,1.000000,1.362500,1.062500,0.362500,22.792998,9.539607,4.861939,6.662133,10.000000
75%,2.000000,1.537500,1.200000,0.412500,32.786197,14.273973,7.200773,9.355335,11.000000
max,2.000000,2.037500,1.625000,2.825000,80.101512,42.184056,21.545620,28.491248,29.000000


In [121]:
crab['Height'].sort_values(ascending=False)

2256    2.8250
748     1.2875
772     0.6250
1485    0.6250
2588    0.6250
         ...  
3542    0.0375
1123    0.0375
1330    0.0250
3867    0.0000
269     0.0000
Name: Height, Length: 3893, dtype: float64

In [122]:
#crab['Height'] = 
crab = crab[crab['Height'] < 1] 
crab = crab[crab['Height'] > 0]

In [123]:
crab['Weight'].sort_values(ascending=False).tail(10)

3633    0.496116
2765    0.439417
1134    0.425243
955     0.411068
1123    0.411068
307     0.396893
3728    0.368544
1177    0.297670
753     0.226796
1330    0.056699
Name: Weight, dtype: float64

In [124]:
#crab = crab[crab['Weight'] > 0.1]

In [125]:
crab.head()

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,1,1.4375,1.1750,0.4125,24.635715,12.332033,5.584852,6.747181,9
1,2,0.8875,0.6500,0.2125,5.400580,2.296310,1.374951,1.559222,6
2,0,1.0375,0.7750,0.2500,7.952035,3.231843,1.601747,2.764076,6
3,1,1.1750,0.8875,0.2500,13.480187,4.748541,2.282135,5.244657,10
4,0,0.8875,0.6625,0.2125,6.903103,3.458639,1.488349,1.700970,6


In [126]:
crab.shape

(3889, 9)

In [134]:
crab['baseline'] = crab['Age'].mean()

In [136]:
base_rmse = np.mean((crab.Age - crab.baseline) ** 2) ** .5
base_rmse

3.2212788357991142

In [135]:
crab.head()

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age,baseline
0,1,1.4375,1.1750,0.4125,24.635715,12.332033,5.584852,6.747181,9,9.956801
1,2,0.8875,0.6500,0.2125,5.400580,2.296310,1.374951,1.559222,6,9.956801
2,0,1.0375,0.7750,0.2500,7.952035,3.231843,1.601747,2.764076,6,9.956801
3,1,1.1750,0.8875,0.2500,13.480187,4.748541,2.282135,5.244657,10,9.956801
4,0,0.8875,0.6625,0.2125,6.903103,3.458639,1.488349,1.700970,6,9.956801


In [128]:
X = crab.drop(columns='Age')
y = crab['Age']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [129]:
lr = LinearRegression()
lr.fit(X_train, y_train)

#lr.score(X_train, y_train), lr.score(X_test, y_test)

LinearRegression()

In [130]:
ss = StandardScaler()
X_train_ss = ss.fit_transform(X_train)
X_test_ss = ss.transform(X_test)

In [131]:
lr = LinearRegression()
knc = KNeighborsRegressor()
dtc = DecisionTreeRegressor()
bag = BaggingRegressor()
rfc = RandomForestRegressor()
abc = AdaBoostRegressor()
svr = svm.SVR()
gbr = GradientBoostingRegressor()


list_of_classifiers = [lr, knc, dtc, bag, rfc, abc, svr, gbr]

In [132]:
for each in list_of_classifiers:
    print(str(each).split('(')[0])
    each.fit(X_train_ss, y_train)
    print('Training score:')
    print(each.score(X_train_ss, y_train))
    print('Testing score:')
    print(each.score(X_test_ss, y_test))
    print()

LinearRegression
Training score:
0.548307456540662
Testing score:
0.5109005634270476

KNeighborsRegressor
Training score:
0.6647229937366266
Testing score:
0.45947941906204315

DecisionTreeRegressor
Training score:
1.0
Testing score:
0.10761192545184128

BaggingRegressor
Training score:
0.9085247619550897
Testing score:
0.48665255563316767

RandomForestRegressor
Training score:
0.9380304998428111
Testing score:
0.5161616397760369

AdaBoostRegressor
Training score:
0.28592603615023815
Testing score:
0.16538022287050158

SVR
Training score:
0.5612030775750558
Testing score:
0.5324880062231494

GradientBoostingRegressor
Training score:
0.6729889267336494
Testing score:
0.5183922227737887



In [133]:
for each in list_of_classifiers:
    print(str(each).split('(')[0])
    pred = each.predict(X_train_ss)
    print('Train RMSE: ')
    print(sqrt(mean_squared_error(y_train, pred)))
    print('Testing RMSE: ')
    pred = each.predict(X_test_ss)
    print(sqrt(mean_squared_error(y_test, pred)))
    print()

LinearRegression
Train RMSE: 
2.1918158527556164
Testing RMSE: 
2.166796294615213

KNeighborsRegressor
Train RMSE: 
1.8883586764922802
Testing RMSE: 
2.277852615631792

DecisionTreeRegressor
Train RMSE: 
0.0
Testing RMSE: 
2.926822479290917

BaggingRegressor
Train RMSE: 
0.9863575591883003
Testing RMSE: 
2.2198580549604205

RandomForestRegressor
Train RMSE: 
0.8118421574509053
Testing RMSE: 
2.1551110405550573

AdaBoostRegressor
Train RMSE: 
2.755841286883015
Testing RMSE: 
2.8305044287482612

SVR
Train RMSE: 
2.1603016111918043
Testing RMSE: 
2.1184386017243257

GradientBoostingRegressor
Train RMSE: 
1.864935571042451
Testing RMSE: 
2.1501375745040177

